<a href="https://colab.research.google.com/github/tanthongtan/ptm/blob/master/sam_geodesic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tanthongtan/ptm

!python ptm/geodesic_sam.py

Cloning into 'ptm'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 27 (delta 2), reused 24 (delta 2), pack-reused 0
Unpacking objects: 100% (27/27), done.
Traceback (most recent call last):
  File "ptm/geodesic_sam.py", line 22, in <module>
    data_tr, tensor_tr, vocab, vocab_size, num_tr = dataset.load_20news_diff()
  File "/content/ptm/dataset.py", line 41, in load_20news_diff
    with open('data/20News-diff/diff.train.tfidf1.data') as file:
FileNotFoundError: [Errno 2] No such file or directory: 'data/20News-diff/diff.train.tfidf1.data'
